## Correlation coefficent vs accuracy

In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
pd.options.mode.chained_assignment = None  
from sklearn.svm import SVR
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error

#random state for division of dataset
#Vary this to 3,4,5,6,7 to reproduce results in the paper
randomstate = 3 

# Drop columns with correlation above defined threshold
def drop_correlated_cols(df, cols_features, threshold):
    cols_to_remove = set()
    correlations_abs = df[cols_features].corr().abs()
    for i in range(len(correlations_abs.columns)):
        for j in range(i):
            if (correlations_abs.iloc[i, j] >= threshold) and (correlations_abs.columns[j] not in cols_to_remove):
                colname = correlations_abs.columns[i]
                if colname in df.columns:
                    cols_to_remove.add(colname)
                    df.drop(colname, axis=1, inplace=True)
    return df

for i in range(0,100,5):
    
    print(i/100)
    
    # Load the entire set
    df = pd.read_excel('Thermo_data_cyclic_species.xlsx', na_values=['na', 'nan'], index_col=0)
    print(df.shape)

    # Drop columns with na
    df.dropna(axis='columns', inplace=True)
    print(df.shape)

    # Drop columns that have only a single value (variance = 0)
    count_unique = df.apply(pd.Series.nunique)
    cols_to_drop = count_unique[count_unique == 1].index
    df.drop(columns=cols_to_drop, inplace=True)
    print(df.shape)

    # Drop correlated features
    cols_features = df.columns[2:]
    df = drop_correlated_cols(df, cols_features, i/100)
    print(df.shape)

    target = df['Enthalpy(KCal)']
    features = df[df.columns[2:]]
    
    #split the dataset
    X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=randomstate, test_size=0.1)
    dfpre=X_train
    dfprenon=X_train
    Testing=X_test

    #Training Normalization
    numerical=list(dfpre)
    scaler = MinMaxScaler()
    dfpre[numerical] = scaler.fit_transform(dfpre[numerical])
    X_train=dfpre

    #Testing Normalization
    test = scaler.transform(Testing)
    X_test=test

    # Fit regression model
    # Define search space
    Cs = [1, 10, 100, 500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 12500, 15000, 20000, 50000]
    epsilons = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

    # Setup the grid to be searched over
    param_grid = dict(C=Cs, epsilon=epsilons)

    y_poly=GridSearchCV(SVR(kernel='rbf'), param_grid, cv=KFold(n_splits=10, shuffle =True, random_state=randomstate), n_jobs=1,verbose=1,scoring='neg_mean_squared_error')
    y_poly.fit(X_train, y_train)
    y_pred_SVR= y_poly.predict(X_test)


    error_ma = mean_absolute_error(y_test, y_pred_SVR)

    print(error_ma)

0.05
(192, 5272)
(192, 5074)
(192, 2480)
(192, 6)
Fitting 10 folds for each of 162 candidates, totalling 1620 fits


[Parallel(n_jobs=1)]: Done 1620 out of 1620 | elapsed:   17.3s finished


9.91748621645
0.1
(192, 5272)
(192, 5074)
(192, 2480)
(192, 8)
Fitting 10 folds for each of 162 candidates, totalling 1620 fits


[Parallel(n_jobs=1)]: Done 1620 out of 1620 | elapsed:   23.1s finished


10.4044571948
0.15
(192, 5272)
(192, 5074)
(192, 2480)
(192, 11)
Fitting 10 folds for each of 162 candidates, totalling 1620 fits


...

[Parallel(n_jobs=1)]: Done 1620 out of 1620 | elapsed:   50.1s finished


1.72164008
0.95
(192, 5272)
(192, 5074)
(192, 2480)
(192, 740)
Fitting 10 folds for each of 162 candidates, totalling 1620 fits


[Parallel(n_jobs=1)]: Done 1620 out of 1620 | elapsed:  1.1min finished


1.63117231356
1.0
(192, 5272)
(192, 5074)
(192, 2480)
(192, 1996)
Fitting 10 folds for each of 162 candidates, totalling 1620 fits
1.81675493614


[Parallel(n_jobs=1)]: Done 1620 out of 1620 | elapsed:  2.6min finished


## Reduce the dataset using fianl correlation coefficient = 0.4

In [37]:
import pandas as pd

# Drop columns with correlation above defined threshold
def drop_correlated_cols(df, cols_features, threshold):
    cols_to_remove = set()
    correlations_abs = df[cols_features].corr().abs()
    for i in range(len(correlations_abs.columns)):
        for j in range(i):
            if (correlations_abs.iloc[i, j] >= threshold) and (correlations_abs.columns[j] not in cols_to_remove):
                colname = correlations_abs.columns[i]
                if colname in df.columns:
                    cols_to_remove.add(colname)
                    df.drop(colname, axis=1, inplace=True)
    return df

# Load the entire set
df = pd.read_excel('Thermo_data_cyclic_species.xlsx', na_values=['na', 'nan'], index_col=0)
print(df.shape)

# Drop columns with na
df.dropna(axis='columns', inplace=True)
print(df.shape)

# Drop columns that have only a single value (variance = 0)
count_unique = df.apply(pd.Series.nunique)
cols_to_drop = count_unique[count_unique == 1].index
df.drop(columns=cols_to_drop, inplace=True)
print(df.shape)

# Drop correlated features
cols_features = df.columns[2:]
df = drop_correlated_cols(df, cols_features, 0.4)
print(df.shape)


# Save dataset
df.to_csv('Dataset_processed.csv')

(192, 5272)
(192, 5074)
(192, 2480)
(192, 49)


## K-Fold error estimation

In [6]:
import numpy as np
import pandas as pd
import csv
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
pd.options.mode.chained_assignment = None

# Load dataset
df = pd.read_csv('Dataset_processed.csv', index_col=0)
target = df['Enthalpy(KCal)']
features = df[df.columns[2:]]
print(features.shape)

# Define search space
Cs = [1, 10, 100, 500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 12500, 15000, 20000, 50000]
epsilons = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

# Setup the grid to be searched over
param_grid = dict(C=Cs, epsilon=epsilons)

# Define outer folds
kFolds = KFold(n_splits=10, shuffle=True, random_state=1).split(X=features.values, y=target.values)

# Define inner folds
grid_search = GridSearchCV(SVR(kernel='rbf', gamma = 'auto'), param_grid, cv=KFold(n_splits=10, shuffle=True, random_state=1),
                           n_jobs=19, verbose=1, scoring='neg_mean_squared_error')

# Open results file and write out headers
out_file = open("grid_search_svr.csv", 'w')
wr = csv.writer(out_file, dialect='excel')
headers = ['C', 'epsilon', "r2", "error_ma", "error_ms", "error_rms", "error_mp", "error_max"]
wr.writerow(headers)
out_file.flush()

for index_train, index_test in kFolds:
    # Get train and test splits
    x_train, x_test = features.iloc[index_train].values, features.iloc[index_test].values
    y_train, y_test = target.iloc[index_train].values, target.iloc[index_test].values

    # Apply min max normalization
    scaler = MinMaxScaler().fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    # Fit
    grid_search.fit(x_train, y_train)

    # Get best params
    best_params = grid_search.best_params_

    # Calculate error metrics
    predictions = grid_search.predict(x_test)
    diff = y_test - predictions
    r2 = r2_score(y_test, predictions)
    error_ma = mean_absolute_error(y_test, predictions)
    error_ms = mean_squared_error(y_test, predictions)
    error_rms = np.sqrt(np.mean(np.square(diff)))
    error_mp = np.mean(abs(np.divide(diff, y_test))) * 100
    error_max = np.amax(np.absolute(diff))

    # Write results
    row = [best_params['C'], best_params['epsilon'], r2,
           error_ma, error_ms, error_rms, error_mp, error_max]
    wr.writerow(row)
    out_file.flush()

out_file.close()

(192, 47)
Fitting 10 folds for each of 162 candidates, totalling 1620 fits


[Parallel(n_jobs=19)]: Done  12 tasks      | elapsed:    5.2s
[Parallel(n_jobs=19)]: Done 500 tasks      | elapsed:    9.3s
[Parallel(n_jobs=19)]: Done 1620 out of 1620 | elapsed:   21.6s finished


Fitting 10 folds for each of 162 candidates, totalling 1620 fits


[Parallel(n_jobs=19)]: Done  12 tasks      | elapsed:    6.1s
[Parallel(n_jobs=19)]: Done 485 tasks      | elapsed:   10.3s
[Parallel(n_jobs=19)]: Done 1620 out of 1620 | elapsed:   23.4s finished


Fitting 10 folds for each of 162 candidates, totalling 1620 fits


...

[Parallel(n_jobs=19)]: Done  12 tasks      | elapsed:    5.4s
[Parallel(n_jobs=19)]: Done 500 tasks      | elapsed:    9.5s
[Parallel(n_jobs=19)]: Done 1620 out of 1620 | elapsed:   19.3s finished


Fitting 10 folds for each of 162 candidates, totalling 1620 fits


[Parallel(n_jobs=19)]: Done  12 tasks      | elapsed:    5.3s
[Parallel(n_jobs=19)]: Done 500 tasks      | elapsed:    9.3s
[Parallel(n_jobs=19)]: Done 1620 out of 1620 | elapsed:   20.2s finished


Fitting 10 folds for each of 162 candidates, totalling 1620 fits


[Parallel(n_jobs=19)]: Done  12 tasks      | elapsed:    5.3s
[Parallel(n_jobs=19)]: Done 500 tasks      | elapsed:    9.2s
[Parallel(n_jobs=19)]: Done 1620 out of 1620 | elapsed:   19.9s finished


## Final Model genaration

In [2]:
import pandas as pd
import pickle
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
pd.options.mode.chained_assignment = None

# Load the entire set
df = pd.read_csv('Dataset_processed.csv', index_col=0)
target = df['Enthalpy(KCal)']
features = df[df.columns[2:]]

# Define search space
Cs = [1, 10, 100, 500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 12500, 15000, 20000, 50000]
epsilons = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

# Setup the grid to be searched over
param_grid = dict(C=Cs, epsilon=epsilons)

# Define grid search
grid_search = GridSearchCV(SVR(kernel='rbf', gamma='auto'), param_grid, cv=KFold(n_splits=10, shuffle=True, random_state=2),
                           n_jobs=19, verbose=1, scoring='neg_mean_squared_error')

# Split data in to features and target
x_train = features.values
y_train = target.values

# Apply min max normalization
scaler = MinMaxScaler().fit(x_train)
x_train = scaler.transform(x_train)

# Find best parameters
grid_search.fit(x_train, y_train)
print(grid_search.best_params_)

# Retrain model with best parameters found from grid search
best_params = grid_search.best_params_
model = SVR(kernel='rbf', gamma='auto', C=best_params['C'], epsilon=best_params['epsilon'])
model.fit(x_train, y_train)

# save the model
filename = 'final_SVR_model.sav'
pickle.dump(model, open(filename, 'wb'))

Fitting 10 folds for each of 162 candidates, totalling 1620 fits


[Parallel(n_jobs=19)]: Done  12 tasks      | elapsed:    5.9s
[Parallel(n_jobs=19)]: Done 371 tasks      | elapsed:    9.3s
[Parallel(n_jobs=19)]: Done 1374 tasks      | elapsed:   20.7s
[Parallel(n_jobs=19)]: Done 1620 out of 1620 | elapsed:   26.8s finished


{'C': 2000, 'epsilon': 0.3}


## Sensitivty Analysis

In [3]:
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model


def get_sensitivity_scores(model, features, top_n):
    # Get just the values of features
    x_train = features.values
    # Apply min max normalization
    scaler = MinMaxScaler().fit(x_train)
    x_train = scaler.transform(x_train)
    # Find mean and standard deviation of each feature
    x_train_avg = np.mean(x_train, axis=0).reshape(1, -1)
    x_train_std = np.std(x_train, axis=0).reshape(1, -1)
    prediction_mean = model.predict(x_train_avg)

    scores_max = []
    # Iterate over each feature
    for i in range(x_train_avg.shape[1]):
        # Copy x_train_avg
        x_train_i = x_train_avg.copy()
        # Add the standard deviation of i to that column
        x_train_i[:, i] = x_train_i[:, i] + (0.1*x_train_std[:, i])
        result_i = model.predict(x_train_i)
        # Take the difference and divide by standard deviation
        diff = (result_i - prediction_mean) / (0.1*x_train_std[:, i])
        scores_max.append(diff.flatten()[0])
    scores_max = np.absolute(scores_max)
    indices_top = np.argsort(scores_max)[-top_n:]
    features_top = features.iloc[:, indices_top].columns
    scores_top = scores_max[indices_top]
    return features_top, scores_top


# Initialize result sheet
df_sensitivity = pd.DataFrame()
n_top = 47

# Load dataset
df = pd.read_csv('Dataset_processed.csv', index_col=0)
features = df[df.columns[2:]]

# Load SVR model
filename = 'final_SVR_model.sav'
model = pickle.load(open(filename, 'rb'))

features_top, scores_top = get_sensitivity_scores(model, features, n_top)
df_sensitivity['SVR features'] = features_top
df_sensitivity['SVR sensitivity values (abs)'] = scores_top

df_sensitivity.to_csv('sensitivity_analysis.csv')

Using TensorFlow backend.
